# 1. Подгружаем текстовый файл *bulgakov_sobache-serdce.txt*

In [ ]:
with open('bulgakov_sobache-serdce.txt') as text:
  book = text.read()

# 2. Работаем со Spacy 

In [ ]:
# Здесь мы устанавливаем нлп модель для русского языка, тк она по умолчанию не входит в пак спейси
# После этого нужно нажать сверху на кнопку runtime и выбрать restart runtime
!pip install https://github.com/explosion/spacy-models/releases/download/ru_core_news_sm-3.1.0/ru_core_news_sm-3.1.0.tar.gz

ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 319, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 104, in resolve
    req, requested_extras=()
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 434, in make_requirement_from_install_req
    version=None,
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/factory.py", line 205, in _make_candidate_from_link
    version=version,
  File "/usr/local/lib/python3.7/dist-packages/p

In [ ]:
import spacy

In [ ]:
import csv

In [ ]:
nlp = spacy.load("ru_core_news_sm")

In [ ]:
nlp.max_length = 20000000

In [ ]:
doc_spacy = nlp(book)

In [ ]:
header = ['spacy', 'start']
rows = []

In [ ]:
for ent in doc_spacy.ents:
  if ent.label_ == 'LOC':
    one_row = []
    one_row = [ent.lemma_, ent.start_char]
    rows.append(one_row)

In [ ]:
rows

In [ ]:
with open('results_spacy.csv','w') as file_spacy:
  writer = csv.writer(file_spacy)
  writer.writerow(header)
  writer.writerows(rows)

# 3. Работаем с Natasha

In [ ]:
!pip install natasha

In [ ]:
import natasha

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    AddrExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [ ]:
doc_natasha = Doc(book)

In [ ]:
doc_natasha.segment(segmenter)

In [ ]:
doc_natasha.tag_morph(morph_tagger)
doc_natasha.parse_syntax(syntax_parser)

In [ ]:
doc_natasha.tag_ner(ner_tagger)

In [ ]:
locations = []

In [ ]:
for span in doc_natasha.spans:
  if span.type == 'LOC':
    locations.append(span)

In [ ]:
for span in locations:
  span.normalize(morph_vocab)

In [ ]:
header_natasha = ['natasha', 'start']
rows_natasha =[]

In [ ]:
for span in locations:
  one_row = []
  one_row = [span.normal, span.start]
  rows_natasha.append(one_row)

In [ ]:
with open('results_natasha.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(header_natasha)
    writer.writerows(rows_natasha)

# 4. Объединяем две таблички (outer) для сравнения


In [ ]:
import pandas as pd

In [ ]:
natasha_table = pd.read_csv("results_natasha.csv")

In [ ]:
natasha_table

In [ ]:
spacy_table = pd.read_csv("results_spacy.csv")

In [ ]:
spacy_table

,spacy,start
0,неглинном,1295
1,сокольники,1605
2,пречистенка,2869
3,нормальный питание,3191
4,мёртвый переулок,10390
...,...,...
94,зине,161510
95,преображенского,163171
96,обуховский переулок,163194
97,борменталя,164422


In [ ]:
merged_tables_outer = pd.merge(natasha_table, spacy_table, on= "start", how = "outer")

In [ ]:
merged_tables_outer

In [ ]:
column_names = ["start", "natasha", "spacy"]

In [ ]:
outer_table = merged_tables_outer.reindex(columns = column_names)

In [ ]:
outer_table.to_csv("outer_table.csv", index=False)

# 5. Объединяем таблички (inner)

In [ ]:
merged_tables_inner = pd.merge(natasha_table, spacy_table, on= "start", how = "inner")

In [ ]:
merged_tables_inner

In [ ]:
inner_table = merged_tables_inner.reindex(columns = column_names)

In [ ]:
inner_table.to_csv("inner_table.csv", index=False)

# 6. Подгружаем черный список 


Черный список можно найти, пройдя по следующей ссылке https://docs.google.com/spreadsheets/d/1sIDbroZOvmGycBQSDh0XTTT-anLNU4zI_XgscgEnGTg/edit?usp=drive_web&ouid=104448250514343948884 

In [ ]:
with open('black_list.csv', 'r') as read_obj:
  b_list = csv.reader(read_obj)
  list_of_csv = list(b_list)
  print(list_of_csv)

[['Абаза'], ['Абакан'], ['Абдулино'], ['Абинск'], ['Агидель'], ['Агрыз'], ['Адыгейск'], ['Азнакаево'], ['Азов'], ['Ак-Довурак'], ['Аксай'], ['Алагир'], ['Алапаевск'], ['Алатырь'], ['Алдан'], ['Алейск'], ['Александров'], ['Александровск'], ['Александровск-Сахалинский'], ['Алексеевка'], ['Алексин'], ['Алзамай'], ['Алупкане призн.'], ['Алуштане призн.'], ['Альметьевск'], ['Амурск'], ['Анадырь'], ['Анапа'], ['Ангарск'], ['Андреаполь'], ['Анжеро-Судженск'], ['Анива'], ['Апатиты'], ['Апрелевка'], ['Апшеронск'], ['Арамиль'], ['Аргун'], ['Ардатов'], ['Ардон'], ['Арзамас'], ['Аркадак'], ['Армавир'], ['Армянскне призн.'], ['Арсеньев'], ['Арск'], ['Артём'], ['Артёмовск'], ['Артёмовский'], ['Архангельск'], ['Асбест'], ['Асино'], ['Астрахань'], ['Аткарск'], ['Ахтубинск'], ['Ачинск'], ['Аша'], ['Бабаево'], ['Бабушкин'], ['Бавлы'], ['Багратионовск'], ['Байкальск'], ['Баймак'], ['Бакал'], ['Баксан'], ['Балабаново'], ['Балаково'], ['Балахна'], ['Балашиха'], ['Балашов'], ['Балей'], ['Балтийск'], ['Бараб

In [ ]:
black_list = []
for i in list_of_csv:
  loc = ''.join(i).lower().split()
  loc = ''.join(loc)
  black_list.append(loc)

# 7. Отсеиваем результаты inner join по черному списку

### 7.1. Начинаем со Spacy

In [ ]:
header_clean_spacy = ['spacy', 'start']
rows_clean_spacy = []

for ent in doc_spacy.ents:
  if ent.label_ == 'LOC':
    if ent.lemma_ not in black_list:
      one_row = []
      one_row = [ent.lemma_, ent.start_char]
      rows_clean_spacy.append(one_row)

In [ ]:
with open('results_spacy_clean.csv','w') as file_spacy_clean:
  writer = csv.writer(file_spacy_clean)
  writer.writerow(header_clean_spacy)
  writer.writerows(rows_clean_spacy)

### 7.2. Проделываем то же самое с кодом Natasha

In [ ]:
header_clean_natasha = ['natasha', 'start']
rows_clean_natasha = []

for span in locations:
  if span.normal.lower() not in black_list:
    one_row = []
    one_row = [span.normal, span.start]
    rows_clean_natasha.append(one_row)

In [ ]:
rows_clean_natasha

In [ ]:
with open('results_natasha_clean.csv', 'w') as file:
  writer = csv.writer(file)
  writer.writerow(header_clean_natasha)
  writer.writerows(rows_clean_natasha)

### 7.3 Объединяем две таблички 

In [ ]:
natasha_table_clean = pd.read_csv("results_natasha_clean.csv")

In [ ]:
spacy_table_clean = pd.read_csv("results_spacy_clean.csv")

In [ ]:
clean_tables_inner = pd.merge(natasha_table_clean, spacy_table_clean, on= "start", how = "inner")

In [ ]:
clean_tables_inner

,natasha,start,spacy
0,Сокольники,1605,сокольники
1,Мертвый переулок,10390,мёртвый переулок
2,Обухов,11389,обухов
3,Мясницкая улица,14254,мясницкий улица
4,Мохова,14742,мохов
5,Рю де ла Пэ,25838,рю де ла пэ
6,Славянский Базар,43538,славянский базар
7,Смоленский,55565,смоленский
8,Преображенскому,78670,преображенский
9,Обуховом переулок,82233,обуховом переулок


In [ ]:
clean_tables_inner.to_csv("clean_results.csv", index=False)

### 7.4. Сравниваем результы двух таблиц *inner_table.txt* и *clean_results.txt*

In [ ]:
merge_new = pd.merge(inner_table, clean_tables_inner, on = 'start', how = 'outer')

In [ ]:
merge_new

,start,natasha_x,spacy_x,natasha_y,spacy_y
0,1605,Сокольники,сокольники,Сокольники,сокольники
1,10390,Мертвый переулок,мёртвый переулок,Мертвый переулок,мёртвый переулок
2,11389,Обухов,обухов,Обухов,обухов
3,13597,Москва,москва,NaN,NaN
4,13917,Москва,москва,NaN,NaN
5,14254,Мясницкая улица,мясницкий улица,Мясницкая улица,мясницкий улица
6,14742,Мохова,мохов,Мохова,мохов
7,14916,Москва,москва,NaN,NaN
8,20452,Гренады,гренада,NaN,NaN
9,25828,Париж,париж,NaN,NaN


In [ ]:
tables = merge_new.rename(columns={'natasha_x': 'natasha', 'spacy_x': 'spacy', 'natasha_y': 'natasha_clean', 'spacy_y': 'spacy_clean'})

In [ ]:
tables

,start,natasha,spacy,natasha_clean,spacy_clean
0,1605,Сокольники,сокольники,Сокольники,сокольники
1,10390,Мертвый переулок,мёртвый переулок,Мертвый переулок,мёртвый переулок
2,11389,Обухов,обухов,Обухов,обухов
3,13597,Москва,москва,NaN,NaN
4,13917,Москва,москва,NaN,NaN
5,14254,Мясницкая улица,мясницкий улица,Мясницкая улица,мясницкий улица
6,14742,Мохова,мохов,Мохова,мохов
7,14916,Москва,москва,NaN,NaN
8,20452,Гренады,гренада,NaN,NaN
9,25828,Париж,париж,NaN,NaN


In [ ]:
tables.to_csv("both_results.csv", index=False)